In [1]:
import nltk
import os
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples

In [2]:
nltk.download('twitter_samples')

[nltk_data] Error loading twitter_samples: <urlopen error Remote end
[nltk_data]     closed connection without response>


False

In [3]:
for name in twitter_samples.fileids():
    print(name)

negative_tweets.json
positive_tweets.json
tweets.20150430-223406.json


In [4]:
negatives = twitter_samples.strings('negative_tweets.json')
positives = twitter_samples.strings('positive_tweets.json')
all_tweets = twitter_samples.strings('tweets.20150430-223406.json')

convert data to dataframe

In [5]:

df_positives = pd.DataFrame(positives)
df_negatives = pd.DataFrame(negatives)
df_positives['label'] = 1
df_negatives['label'] = -1
df = pd.concat([df_positives , df_negatives] , ignore_index= True)
df = df.sample(frac = 1).reset_index(drop = True)

In [6]:
df.head(20)

,0,label
0,SICK :(,-1
1,@gadventures So hard to pick just one. The Inc...,-1
2,@KayOddone Kinda smug to know we were doing al...,1
3,@LaraKBaker Great view! It's a shame about the...,-1
4,I feel like crying for no reasons right now :(,-1
5,"@z1mb0bw4y and 2 weeks of vacation, which I do...",-1
6,Mum asked me if I wanted to go to the bookstor...,1
7,im still giddy over d1 :-(,-1
8,something wrong :(,-1
9,prelims exam tomorrow :-) #costacc @AcctncyPr...,1


preparing dataset

In [7]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [8]:
df[0].iloc[1]

'@gadventures So hard to pick just one. The Inca Trail was tough but fair, sandboarding was fun but lost my iPhone :( http://t.co/98H24VFdp4'

deleting noises

In [9]:
df[0] = df[0].apply(lambda x: re.sub(r'\$\w*', '', x))
df[0] = df[0].apply(lambda x: re.sub(r'^RT[\s]+', '', x))
df[0] = df[0].apply(lambda x: re.sub(r'https?:\/\/.*[\r\n]*', '', x))
df[0] = df[0].apply(lambda x: re.sub(r'#', '', x))


tokenization

In [10]:
df['tokens']= df[0].str.split()
print(df['tokens'])

0                                              [SICK, :(]
1       [@gadventures, So, hard, to, pick, just, one.,...
2       [@KayOddone, Kinda, smug, to, know, we, were, ...
3       [@LaraKBaker, Great, view!, It's, a, shame, ab...
4       [I, feel, like, crying, for, no, reasons, righ...
                              ...                        
9995    [@Gotzefying, Im, trying, to, D/L, MH3, englis...
9996    [@Candiferslaw, Lucky, dog, I, hate, you...., :)]
9997                             [@bbgurrll, i, wish, :(]
9998    [@nnoonlight, i, missed, you, :(, it, was, oka...
9999    [@perfect_st0rms, omg, nooo, you, should, have...
Name: tokens, Length: 10000, dtype: object


deleting stop_words

In [11]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
print(df['tokens'])

0                                              [SICK, :(]
1       [@gadventures, So, hard, pick, one., The, Inca...
2       [@KayOddone, Kinda, smug, know, stuff, Marrick...
3       [@LaraKBaker, Great, view!, It's, shame, weath...
4             [I, feel, like, crying, reasons, right, :(]
                              ...                        
9995    [@Gotzefying, Im, trying, D/L, MH3, english, p...
9996    [@Candiferslaw, Lucky, dog, I, hate, you...., :)]
9997                                [@bbgurrll, wish, :(]
9998    [@nnoonlight, missed, :(, okay., lol., I, noth...
9999       [@perfect_st0rms, omg, nooo, said, hi!, :((((]
Name: tokens, Length: 10000, dtype: object


[nltk_data] Error loading stopwords: <urlopen error Remote end closed
[nltk_data]     connection without response>


stemming

In [12]:
nltk.download('wordnet')
stemmer = PorterStemmer()
df['stemmed'] = df['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

[nltk_data] Error loading wordnet: <urlopen error Remote end closed
[nltk_data]     connection without response>


In [13]:
print(df['tokens'] , df['stemmed'])

0                                              [SICK, :(]
1       [@gadventures, So, hard, pick, one., The, Inca...
2       [@KayOddone, Kinda, smug, know, stuff, Marrick...
3       [@LaraKBaker, Great, view!, It's, shame, weath...
4             [I, feel, like, crying, reasons, right, :(]
                              ...                        
9995    [@Gotzefying, Im, trying, D/L, MH3, english, p...
9996    [@Candiferslaw, Lucky, dog, I, hate, you...., :)]
9997                                [@bbgurrll, wish, :(]
9998    [@nnoonlight, missed, :(, okay., lol., I, noth...
9999       [@perfect_st0rms, omg, nooo, said, hi!, :((((]
Name: tokens, Length: 10000, dtype: object 0                                              [sick, :(]
1       [@gadventur, so, hard, pick, one., the, inca, ...
2       [@kayoddon, kinda, smug, know, stuff, marrickv...
3       [@larakbak, great, view!, it', shame, weather,...
4                 [i, feel, like, cri, reason, right, :(]
                             

In [16]:
tfidf = TfidfVectorizer()
df['stemmed'] = df['stemmed'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
x_tfidf = tfidf.fit_transform(df['stemmed'])
x_tfidf = x_tfidf.toarray()
label = df['label']

trainx, valx, trainy, valy = train_test_split(x_tfidf, label, test_size=0.2, random_state=42)

trainy = trainy.values.reshape((8000,1))
valy = valy.values.reshape((2000,1))


In [15]:
valx.shape

(2000, 17878)

In [17]:
valy.shape

(2000, 1)

Lemmatization

In [18]:
def initialization(dim):
    w = np.zeros((dim,1))
    b = 0

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w,b

In [19]:
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

In [20]:
def propagate(w, x, b, y):
    m = x.shape[1]
    A = sigmoid(np.dot(w.T,x) + b) 
    cost = cost = np.sum(((- np.log(A))*y + (-np.log(1-A))*(1-y)))/m  

    dw = (np.dot(x,(A-y).T))/m
    db = (np.sum(A-y))/m
    backs = {"dW" : dw,
             "dB" : db}
    return cost, backs

In [25]:
def optimize(w, b, x, y, learning_rate, num_iteration):

    costs = []
    for i in range(num_iteration):
        cost, backs = propagate(w, x, b, y)
        dw = backs["dW"]
        db = backs["dB"]
        w = w - learning_rate*dw
        b = b - learning_rate*db   

        if i%100 == 0:
            costs.append(cost)

        params = {"w": w,
                  "b": b} 
        grads = {"dw": dw,
                 "db": db}   
    return costs, params, grads

In [26]:
def predict(w, x, b):

    m = x.shape[1]
    y_prediction = np.zeros((1,m))
    A = sigmoid(np.dot(w.T , x) + b)
    y_prediction = (A >= 0.5) * 1.0
    assert(y_prediction.shape == (1,m))

    return y_prediction

In [27]:
def model(X_train, y_train, X_test, y_test, learning_rate, num_iteration):
    
    w, b = initialization((X_train.shape[0]))
    costs, params, grads = optimize(w, b, X_train, y_train, learning_rate, num_iteration)

    w = params["w"]
    b = params["b"]

    y_prediction_train = predict(w, X_train, b)
    y_prediction_test = predict(w, X_test, b)

    print("train accuracy : ".fortmat(100 - np.mean(np.abs(y_train - y_prediction_train))*100))
    print("test accuracy : ".fortmat(100 - np.mean(np.abs(y_test - y_prediction_test))*100))

    d = {"costs" : costs,
         "y_prediction_test": y_prediction_test, 
         "y_prediction_train" : y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iteration": num_iteration}
    
    return d

In [ ]:
d = model(trainx, trainy, valx, valy, learning_rate = 0.005,  num_iteration = 2000)

In [262]:
valx.shape

(2000, 17878)